<a href="https://colab.research.google.com/github/lucia1299/DBMS_InvestigativeJournalism/blob/main/bio_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xmltodict

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset, concatenate_datasets, Features, Sequence, ClassLabel, Value
import os
import xmltodict
import random
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def parse_xml_files(word_directory, attribute_directory):
    dict_pairs = []
    for word_filename in os.listdir(word_directory):
        if word_filename.endswith(".xml"):
            with open(os.path.join(word_directory, word_filename), 'r') as file:
                xml_string = file.read()
                word_dict = xmltodict.parse(xml_string)
                for attribute_filename in os.listdir(attribute_directory):
                    if attribute_filename.endswith(".xml") and attribute_filename == word_filename:
                        with open(os.path.join(attribute_directory, attribute_filename), 'r') as file:
                            xml_string = file.read()
                            attribute_dict = xmltodict.parse(xml_string)
                            dict_pairs.append((word_dict, attribute_dict))
    return dict_pairs

dict_pairs=parse_xml_files('/content/drive/MyDrive/TESI/tryout/Basedata', '/content/drive/MyDrive/TESI/tryout/Markable')
print(len(dict_pairs))
print(dict_pairs[1][0])

50
{'words': {'word': [{'@id': 'word_1', '#text': 'A'}, {'@id': 'word_2', '#text': 'Fernandez'}, {'@id': 'word_3', '#text': 'il'}, {'@id': 'word_4', '#text': '50%'}, {'@id': 'word_5', '#text': 'delle'}, {'@id': 'word_6', '#text': "attivita'"}, {'@id': 'word_7', '#text': 'brasiliane'}, {'@id': 'word_8', '#text': '.'}, {'@id': 'word_9', '#text': '('}, {'@id': 'word_10', '#text': 'NOSTRO'}, {'@id': 'word_11', '#text': 'SERVIZIO'}, {'@id': 'word_12', '#text': ')'}, {'@id': 'word_13', '#text': 'BOSTON'}, {'@id': 'word_14', '#text': '_'}, {'@id': 'word_15', '#text': 'Dopo'}, {'@id': 'word_16', '#text': 'la'}, {'@id': 'word_17', '#text': 'joint-venture'}, {'@id': 'word_18', '#text': 'in'}, {'@id': 'word_19', '#text': 'Cina'}, {'@id': 'word_20', '#text': ','}, {'@id': 'word_21', '#text': 'Benetton'}, {'@id': 'word_22', '#text': 'torna'}, {'@id': 'word_23', '#text': 'a'}, {'@id': 'word_24', '#text': 'tessere'}, {'@id': 'word_25', '#text': 'dalle'}, {'@id': 'word_26', '#text': 'Americhe'}, {'@id

In [ ]:
#create a list of dictionaries containing word ids and correponding tags

def create_single_markable(attribute_dict):
    result_dict = {}
    container_list = []

    for markable in attribute_dict['markables']['markable']:
        id = markable['@id']
        span = markable['@span']
        attrib_role = markable['@attribution_role']
        sub_span = span.split(',')
        for sub in sub_span:
            if '..' in sub:
                start, end = [int(i.split('_')[1]) for i in sub.split('..')]
                words = [f'word_{i}' for i in range(start, end+1)]
                result_dict['word_id'] = words
                attrib_list = []
                for word in words:
                    attrib_list.append(attrib_role)
                result_dict['tag'] = attrib_list
                container_list.append(result_dict)
                result_dict = {}
            elif ',' in sub and '..' not in sub:
                words = sub.split(',')
                result_dict['word_id'] = words
                attrib_list = [attrib_role for _ in words]
                result_dict['tag'] = attrib_list
                container_list.append(result_dict)
                result_dict={}
            else:
                result_dict['word_id'] = [span]
                result_dict['tag'] = [attrib_role]
                container_list.append(result_dict)
                result_dict = {}

    return container_list

In [ ]:
attribute_dict=dict_pairs[1][1]
container_list = create_single_markable(attribute_dict)
container_list[2]

{'word_id': ['word_76', 'word_77', 'word_78', 'word_79'],
 'tag': ['source', 'source', 'source', 'source']}

In [ ]:
#check which markable dictionaries are empty and therefore raise an error message: markable

attribute_dicts=[]
main_list = []

for x in dict_pairs:
    attribute_dicts.append(x[1])

print(len(attribute_dicts))

for i, attribute_dict in enumerate(attribute_dicts):
    try:
        container_list = create_single_markable(attribute_dict)
        main_list.append(container_list)
    except Exception as e:
        print(f"An error occurred at index {i} of attribute_dicts. Error message: {str(e)}")

50
An error occurred at index 4 of attribute_dicts. Error message: 'markable'
An error occurred at index 13 of attribute_dicts. Error message: 'markable'
An error occurred at index 15 of attribute_dicts. Error message: 'markable'
An error occurred at index 23 of attribute_dicts. Error message: 'markable'
An error occurred at index 25 of attribute_dicts. Error message: 'markable'
An error occurred at index 34 of attribute_dicts. Error message: 'markable'
An error occurred at index 43 of attribute_dicts. Error message: 'markable'


In [ ]:
# create a list of dictionaries in which there is a sentence id, the word ids and the actual words in that sentence

def create_wordID_list(word_dict):
    list_words_dicts=[]

    words_list = word_dict['words']['word']
    random_number = random.randint(1, 10000)

    sentence = []
    word_ids = []
    for idx, word in enumerate(words_list):
        sentence.append(word['#text'])
        word_ids.append(word['@id'])
        if word['#text'] == '.':
            other_dict={}
            other_dict['sentence_id']=[str(random_number)]
            other_dict['tokens']=sentence
            other_dict['words_ids'] = word_ids
            list_words_dicts.append(other_dict)
            sentence = []
            word_ids = []

    return list_words_dicts

In [ ]:
word_dict = dict_pairs[1][0]
list_words_dicts = create_wordID_list(word_dict)
list_words_dicts[0]

{'sentence_id': ['6822'],
 'tokens': ['A',
  'Fernandez',
  'il',
  '50%',
  'delle',
  "attivita'",
  'brasiliane',
  '.'],
 'words_ids': ['word_1',
  'word_2',
  'word_3',
  'word_4',
  'word_5',
  'word_6',
  'word_7',
  'word_8']}

In [ ]:
# check to see whether there there are any files that raise an issue in the word dict

main_list_2 = []
word_dicts=[]
for x in dict_pairs:
    word_dicts.append(x[0])

print(len(word_dicts))

for i, word_dict in enumerate(word_dicts):
    try:
        list_words_dicts = create_wordID_list(word_dict)
        main_list_2.append(list_words_dicts)
    except Exception as e:
        print(f"An error occurred at index {i} of word_dicts. Error message: {str(e)}")

50


In [ ]:
# def add_tags(container_list, list_words_dicts):

#     for markable_dict in list_words_dicts:
#         word_ids = markable_dict['words_ids']
#         new_tag_list=[]
#         for id in word_ids:
#             for word_dict in container_list:
#                 tags_list=word_dict['tag']
#                 word_ids_2=word_dict['word_id']
#                 if id in word_ids_2:
#                     corresponding_tag=tags_list[0]
#                     new_tag_list.append(corresponding_tag)
#                 else:
#                     new_tag_list.append('O')
#                 break
#         markable_dict['tag']=new_tag_list

#     return list_words_dicts

In [ ]:
def add_tags(container_list, list_words_dicts):
    for markable_dict in list_words_dicts:
        word_ids = markable_dict['words_ids']
        new_tag_list = []
        for i, id in enumerate(word_ids):
            for word_dict in container_list:
                tags_list = word_dict['tag']
                word_ids_2 = word_dict['word_id']
                if id in word_ids_2:
                    corresponding_tag = tags_list[0]
                    if i == 0:
                        corresponding_tag = 'B-' + corresponding_tag
                    else:
                        corresponding_tag = 'I-' + corresponding_tag
                    new_tag_list.append(corresponding_tag)
                else:
                    new_tag_list.append('O')
                break
        markable_dict['tag'] = new_tag_list
    return list_words_dicts

In [ ]:
list_enriched = add_tags(container_list, list_words_dicts)
list_enriched[0]

{'sentence_id': ['7425'],
 'tokens': ['I',
  'colpiti',
  'sono',
  'uomini',
  'di',
  'Bardellino',
  ',',
  're',
  'del',
  'traffico',
  'di',
  'cocaina',
  '.'],
 'words_ids': ['word_1',
  'word_2',
  'word_3',
  'word_4',
  'word_5',
  'word_6',
  'word_7',
  'word_8',
  'word_9',
  'word_10',
  'word_11',
  'word_12',
  'word_13'],
 'tag': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}

In [ ]:
def tags_exception(list_words_dicts):

    for word_dict in list_words_dicts:
        word_list = word_dict['tokens']
        attr_list=[]

        for word in word_list:
            attr_list.append('O')
        word_dict['tag'] = attr_list

    return list_words_dicts

In [ ]:
new_list_words = tags_exception(list_words_dicts)
new_list_words[1]

{'sentence_id': ['7425'],
 'tokens': ['GUERRA',
  'PER',
  'BANDE',
  'NEL',
  'NAPOLETANO',
  'TRE',
  'UCCISI',
  ',',
  'GRAVISSIMO',
  'UN',
  'PASSANTE',
  '.'],
 'words_ids': ['word_14',
  'word_15',
  'word_16',
  'word_17',
  'word_18',
  'word_19',
  'word_20',
  'word_21',
  'word_22',
  'word_23',
  'word_24',
  'word_25'],
 'tag': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}

In [ ]:
def create_dataset_per_file(enriched_list):

    container_dict = {key: [d[key] for d in enriched_list] for key in enriched_list[0]}
    ds = Dataset.from_dict(container_dict, features=Features({
                    'sentence_id': Sequence(Value(dtype='string')),
                    'words_ids': Sequence(Value(dtype='string')),
                    'tag': Sequence(ClassLabel(names=['B-cue', 'B-source', 'B-supplement', 'B-content', 'I-cue', 'I-source', 'I-supplement', 'I-content','O'])),
                    'tokens': Sequence(Value(dtype='string'))
                    }))

    return ds

In [ ]:
def general_function(word_directory, attribute_directory):
    dataset_list=[]

    dict_pairs=parse_xml_files(word_directory, attribute_directory)

    for i in dict_pairs:
        enriched_list=[]
        word_dict=i[0]
        attribute_dict=i[1]
        list_words_dicts = create_wordID_list(word_dict)
        for dict_val in attribute_dict.values():
            if 'markable' in dict_val:
                container_list = create_single_markable(attribute_dict)
                enriched_list = add_tags(container_list, list_words_dicts)
            else:
                enriched_list = tags_exception(list_words_dicts)

        single_dataset = create_dataset_per_file(enriched_list)
        dataset_list.append(single_dataset)

    main_dataset=concatenate_datasets(dataset_list)

    return main_dataset

In [ ]:
main_dataset = general_function('/content/drive/MyDrive/TESI/tryout/Basedata', '/content/drive/MyDrive/TESI/tryout/Markable')
main_dataset

Dataset({
    features: ['sentence_id', 'tokens', 'words_ids', 'tag'],
    num_rows: 1495
})

In [ ]:
main_dataset[0]

{'sentence_id': ['7600'],
 'tokens': ['SALONE', 'DEL', 'LIBRO', ':', 'AFFLUENZA', 'RECORD', '.'],
 'words_ids': ['word_1',
  'word_2',
  'word_3',
  'word_4',
  'word_5',
  'word_6',
  'word_7'],
 'tag': [8, 8, 8, 8, 8, 8, 8]}

In [ ]:
print(type(main_dataset))

<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
main_dataset.features['tag']

Sequence(feature=ClassLabel(names=['B-cue', 'B-source', 'B-supplement', 'B-content', 'I-cue', 'I-source', 'I-supplement', 'I-content', 'O'], id=None), length=-1, id=None)

In [ ]:
train_test_split_ratio = 0.8  # 80% for training
train_val_split_ratio = 0.75  # 75% of 80% for training, 25% of 80% for validation

# Compute the actual split sizes
test_split = 1 - train_test_split_ratio
val_split = (1 - train_val_split_ratio) * train_test_split_ratio

# Perform the splits
dataset = main_dataset.train_test_split(test_size=test_split)
train_dataset = dataset['train'].train_test_split(test_size=val_split)
test_dataset = dataset['test']


# Now, `train_dataset['train']` is the training set,
# `train_dataset['test']` is the validation set,
# and `test_dataset` is the test set.

test_dataset

Dataset({
    features: ['sentence_id', 'tokens', 'words_ids', 'tag'],
    num_rows: 299
})

In [ ]:
train_dataset['train']

Dataset({
    features: ['sentence_id', 'tokens', 'words_ids', 'tag'],
    num_rows: 956
})

In [ ]:
train_dataset['test']

Dataset({
    features: ['sentence_id', 'tokens', 'words_ids', 'tag'],
    num_rows: 240
})

In [ ]:
from collections import Counter

# Assuming 'dataset' is your Dataset object
tag_counter = Counter()

for row in train_dataset['train']:
    for tag in row['tag']:  # Assuming 'tag' is a list
        tag_counter[tag] += 1

for tag, count in tag_counter.items():
    print(f"Tag: {tag}, Count: {count}")


Tag: 8, Count: 23224
Tag: 5, Count: 5
Tag: 7, Count: 186
Tag: 3, Count: 3
Tag: 4, Count: 14
Tag: 6, Count: 5
Tag: 0, Count: 2


In [ ]:
train_dataset['train'].save_to_disk('/content/drive/MyDrive/TESI/tryout/BIOtraining')
train_dataset['test'].save_to_disk('/content/drive/MyDrive/TESI/tryout/BIOevaluating')
test_dataset.save_to_disk('/content/drive/MyDrive/TESI/tryout/BIOtesting')

Saving the dataset (0/1 shards):   0%|          | 0/956 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/240 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/299 [00:00<?, ? examples/s]